# Multi-Stream Neural Networks: CIFAR-100 Training

This notebook demonstrates the full pipeline for training multi-stream neural networks on CIFAR-100 data:

🚀 **Features:**
- Automatic GPU detection and optimization
- RGB to RGBL preprocessing with visualizations
- BaseMultiChannelNetwork (Dense) and MultiChannelResNetNetwork (CNN) models
- Dynamic progress bars during training
- Comprehensive evaluation and analysis

**Hardware Requirements:**
- Google Colab with GPU runtime (A100/V100 recommended)
- Sufficient memory for CIFAR-100 dataset processing

## 1. Setup: Mount Drive and Navigate to Project

Mount Google Drive and navigate to the existing Multi-Stream Neural Networks project directory.

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Navigate to Drive and project directory
import os
os.chdir('/content/drive/MyDrive')

# Navigate to the existing project (assuming it's already cloned)
project_path = '/content/drive/MyDrive/Multi-Stream-Neural-Networks'
if os.path.exists(project_path):
    os.chdir(project_path)
    print(f"✅ Found project at: {project_path}")
else:
    print(f"❌ Project not found at: {project_path}")
    print("💡 Please clone the repository first:")
    print("   !git clone https://github.com/clingergab/Multi-Stream-Neural-Networks.git")

Cloning into 'Multi-Stream-Neural-Networks'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 228 (delta 33), reused 221 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (228/228), 258.62 KiB | 2.94 MiB/s, done.
Resolving deltas: 100% (33/33), done.


## 2. Update Repository

Pull the latest changes from the repository to ensure you have the most up-to-date code.

In [4]:
# Update repository with latest changes
print("🔄 Pulling latest changes from repository...")

# Make sure we're in the right directory
os.chdir('/content/drive/MyDrive/Multi-Stream-Neural-Networks')
print(f"📁 Current directory: {os.getcwd()}")

# Pull latest changes
!git pull origin main

# Show latest commit info
print("\n📋 Latest commit:")
!git log --oneline -1

# Check status
print("\n📊 Repository status:")
!git status --short

print("\n✅ Repository update complete!")

🔄 Pulling latest changes from repository...


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Multi-Stream-Neural-Networks'

## 3. Install Dependencies and Import Libraries

Install compatible PyTorch/NumPy versions and import all required libraries for the multi-stream neural network training.

In [ ]:
# 🔧 Smart Dependency Setup - Uses Colab defaults when possible
print("🔧 Setting up dependencies with Colab compatibility...")

# Check current environment
import sys
print(f"Python version: {sys.version}")

# First, try to import existing packages (Colab defaults)
print("📦 Checking for existing packages...")

try:
    import numpy as np
    import torch
    import torchvision
    print("✅ Found existing PyTorch installation")
    print(f"   NumPy: {np.__version__}")
    print(f"   PyTorch: {torch.__version__}")
    print(f"   Torchvision: {torchvision.__version__}")
    print(f"   CUDA available: {torch.cuda.is_available()}")
    
    # Check for CUDA version compatibility
    pytorch_cuda = torch.__version__.split('+')[-1] if '+' in torch.__version__ else 'unknown'
    torchvision_cuda = torchvision.__version__.split('+')[-1] if '+' in torchvision.__version__ else 'unknown'
    
    if pytorch_cuda == torchvision_cuda:
        print(f"✅ CUDA versions match: {pytorch_cuda}")
        use_existing = True
    else:
        print(f"⚠️ CUDA version mismatch: PyTorch={pytorch_cuda}, Torchvision={torchvision_cuda}")
        use_existing = False
        
except ImportError as e:
    print(f"⚠️ Missing packages: {e}")
    use_existing = False

# If existing packages work, use them; otherwise install compatible versions
if use_existing:
    print("🎯 Using existing Colab packages - no installation needed!")
else:
    print("📦 Installing compatible package versions...")
    
    # Install specific compatible versions
    !pip install -q "numpy>=1.24.4,<2.1.0"
    !pip install -q torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0+cu118 --index-url https://download.pytorch.org/whl/cu118
    !pip install -q tqdm matplotlib seaborn scikit-learn

# Import all required libraries
print("\n📦 Importing all required libraries...")

import os
import warnings
warnings.filterwarnings('ignore')

# Add project to Python path
sys.path.append('/content/drive/MyDrive/Multi-Stream-Neural-Networks')

# Core libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms

# Data and visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import pickle
from typing import Tuple, Dict, List
import time
from pathlib import Path

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Verify final setup
print("🔍 Final verification...")
print(f"   NumPy: {np.__version__}")
print(f"   PyTorch: {torch.__version__}")
print(f"   Torchvision: {torchvision.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Import project modules
print("\n📁 Importing project modules...")
try:
    from src.models.basic_multi_channel.base_multi_channel_network import BaseMultiChannelNetwork
    from src.models.basic_multi_channel.multi_channel_resnet_network import MultiChannelResNetNetwork
    from src.transforms.rgb_to_rgbl import RGBtoRGBL
    from src.utils.colab_utils import load_cifar10
    from src.utils.cifar100_loader import get_cifar100_datasets, CIFAR100_FINE_LABELS, SimpleDataset
    print("✅ All imports successful!")
except ImportError as e:
    print(f"⚠️ Import warning: {e}")
    print("   Make sure you've updated the repository in the previous step")

# Final compatibility check
print("\n🧪 Testing NumPy-PyTorch compatibility...")
try:
    test_array = np.array([1, 2, 3], dtype=np.float32)
    test_tensor = torch.from_numpy(test_array)
    result = test_tensor + 1
    print("✅ NumPy-PyTorch integration working perfectly!")
    print("✅ Ready to proceed to Step 4 (Load CIFAR-100 Dataset)!")
except Exception as e:
    print(f"⚠️ Minor compatibility issue: {e}")
    print("💡 This usually doesn't affect functionality - you can still proceed!")

print("\n🎯 Setup complete! You can now proceed to Step 4.")

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 126.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 17.0 MB/s e

RuntimeError: Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=12.4. Please reinstall the torchvision that matches your PyTorch install.

## 4. Load CIFAR-100 Dataset

Load the CIFAR-100 dataset and verify its structure.

In [2]:
# Import necessary libraries for data loading
from pathlib import Path
import sys
import os

# Import our utilities
print("📁 Setting up CIFAR-100 dataset loading...")
try:
    from src.utils.cifar100_loader import (
        load_cifar100_raw, 
        get_cifar100_datasets, 
        SimpleDataset
    )
    print("✅ CIFAR-100 loader utilities imported successfully")
except ImportError as e:
    print(f"❌ Failed to import CIFAR-100 utilities: {e}")
    print("💡 Make sure src/utils/cifar100_loader.py exists and is accessible")
    raise

# Check if data folder exists
data_path = Path("data/cifar-100")
if data_path.exists():
    print(f"✅ Data folder found at: {data_path}")
else:
    print(f"⚠️ Data folder not found at: {data_path}")
    print("💡 Will create it when downloading CIFAR-100 data")

# Load the datasets using our utility
print("📊 Loading CIFAR-100 datasets...")
train_dataset, test_dataset, cifar100_fine_labels = get_cifar100_datasets()

# Get raw data for backward compatibility with existing code
train_data = train_dataset.data
train_labels = train_dataset.labels
test_data = test_dataset.data
test_labels = test_dataset.labels

print(f"\n📊 Dataset Info:")
print(f"   Training samples: {len(train_data)}")
print(f"   Test samples: {len(test_data)}")
print(f"   Image shape: {train_data[0].shape}")
print(f"   Number of classes: {len(cifar100_fine_labels)}")
print(f"   Label format: Single integer (fine labels 0-99)")
print(f"   Data range: [{train_data.min():.3f}, {train_data.max():.3f}]")

print("✅ CIFAR-100 datasets ready for processing!")
print("💡 No torchvision naming conventions needed - loaded directly from pickle files!")

# Final compatibility check (simplified to avoid NumPy 2.x issues)
print("\n🧪 Testing NumPy-PyTorch compatibility...")
try:
    # Simple test that should work with NumPy 2.x
    # Use explicit imports to avoid scoping issues
    import numpy
    import torch as pytorch
    
    test_data = [1, 2, 3]
    test_array = numpy.array(test_data, dtype=numpy.float32)
    test_tensor = pytorch.tensor(test_data, dtype=pytorch.float32)
    
    # Test basic operations
    array_sum = test_array.sum()
    tensor_sum = test_tensor.sum()
    
    # Test tensor conversion (this often fails with NumPy 2.x issues)
    converted_tensor = pytorch.from_numpy(test_array.copy())
    
    print("✅ NumPy-PyTorch integration working!")
    print("✅ Ready to proceed with CIFAR-100 loading!")
    
    # Note about NumPy 2.x warnings
    if numpy.__version__.startswith('2.'):
        print("ℹ️ Note: You may see NumPy 2.x compatibility warnings, but they don't affect functionality.")
        
except Exception as e:
    print(f"⚠️ NumPy-PyTorch compatibility issue: {e}")
    print("💡 This is likely a NumPy 2.x scoping issue, but shouldn't affect CIFAR-100 loading")
    print("✅ You can still proceed to Step 4 - the data loading will work fine!")

📁 Setting up CIFAR-100 dataset loading...
❌ Failed to import CIFAR-100 utilities. Make sure src/utils/cifar100_loader.py exists


ImportError: cannot import name 'CIFAR100Loader' from 'src.utils.cifar100_loader' (/Users/gclinger/Documents/projects/Multi-Stream-Neural-Networks/src/utils/cifar100_loader.py)

## 5. Process Data: RGB to Multi-Stream Format

Convert RGB images to both RGB and brightness (luminance) channels to create multi-stream data.

In [ ]:
# RGB to RGBL Data Processing with caching support
print("🔄 Starting RGB to RGBL data processing...")

# Import the transformation (should work if previous cells succeeded)
try:
    from src.transforms.rgb_to_rgbl import RGBtoRGBL
    print("✅ RGBtoRGBL transform imported successfully")
except ImportError as e:
    print(f"❌ Failed to import RGBtoRGBL: {e}")
    print("💡 Make sure you're in the correct directory and have run git pull")
    raise

def convert_dataset_to_multi_stream_batch(dataset, max_samples=None, cache_file=None, force_reprocess=False, batch_size=256):
    """
    Convert a CIFAR-100 dataset to multi-stream format (RGB + Brightness) using efficient batch processing.
    Supports caching to avoid reprocessing if interrupted.

    Args:
        dataset: CIFAR-100 dataset
        max_samples: Maximum number of samples to process (for faster testing)
        cache_file: Path to cache file for resuming processing
        force_reprocess: Force reprocessing even if cache exists
        batch_size: Number of images to process in each batch (default: 256)

    Returns:
        rgb_data: RGB channel data [N, 3, 32, 32]
        brightness_data: Brightness channel data [N, 1, 32, 32]
        labels: Class labels [N]
    """
    # Check for cached results first
    if cache_file and Path(cache_file).exists() and not force_reprocess:
        print(f"✅ Loading cached data from {cache_file}")
        try:
            with open(cache_file, 'rb') as f:
                cached_data = pickle.load(f)
            print(f"   Loaded {len(cached_data['labels'])} samples from cache")
            return cached_data['rgb_data'], cached_data['brightness_data'], cached_data['labels']
        except Exception as e:
            print(f"⚠️ Failed to load cache: {e}. Reprocessing...")

    print(f"🔄 Converting dataset to multi-stream format using batch processing...")
    print(f"   Batch size: {batch_size}")

    # Initialize RGB to RGBL transform
    rgb_to_rgbl = RGBtoRGBL()
    print("✅ RGB to RGBL transform initialized")

    # Determine number of samples to process
    num_samples = len(dataset) if max_samples is None else min(max_samples, len(dataset))
    print(f"   Processing {num_samples} samples...")

    # Initialize arrays
    rgb_data = []
    brightness_data = []
    labels = []

    # Process samples in batches with progress bar
    try:
        num_batches = (num_samples + batch_size - 1) // batch_size
        
        with tqdm(range(0, num_samples, batch_size), desc="Processing batches", total=num_batches) as pbar:
            for batch_start in pbar:
                batch_end = min(batch_start + batch_size, num_samples)
                current_batch_size = batch_end - batch_start
                
                try:
                    # Collect batch data
                    batch_images = []
                    batch_labels = []
                    
                    for i in range(batch_start, batch_end):
                        image, label = dataset[i]
                        
                        # Convert label to int if it's a tensor
                        if hasattr(label, 'item'):
                            label = label.item()
                        
                        batch_images.append(image)
                        batch_labels.append(label)
                    
                    # Stack images into batch tensor [B, C, H, W]
                    batch_tensor = torch.stack(batch_images)
                    
                    # Apply RGBtoRGBL transform to entire batch - it returns a TUPLE (rgb_batch, brightness_batch)
                    rgb_batch, brightness_batch = rgb_to_rgbl(batch_tensor)
                    
                    # Validate the returned tensors
                    if not isinstance(rgb_batch, torch.Tensor) or not isinstance(brightness_batch, torch.Tensor):
                        print(f"⚠️ Warning: RGBtoRGBL returned unexpected types: {type(rgb_batch)}, {type(brightness_batch)}")
                        continue
                    
                    # Validate tensor dimensions
                    if rgb_batch.dim() != 4 or rgb_batch.shape[1] != 3:
                        print(f"⚠️ Warning: Expected RGB batch [B, 3, H, W], got {rgb_batch.shape}")
                        continue
                        
                    if brightness_batch.dim() != 4 or brightness_batch.shape[1] != 1:
                        print(f"⚠️ Warning: Expected brightness batch [B, 1, H, W], got {brightness_batch.shape}")
                        continue
                    
                    # Convert to numpy and add to our data lists
                    rgb_data.append(rgb_batch.numpy())
                    brightness_data.append(brightness_batch.numpy())
                    labels.extend(batch_labels)
                    
                    # Update progress bar
                    pbar.set_postfix({'processed': len(labels), 'batch_size': current_batch_size})

                    # Save checkpoint every few batches (every ~5000 samples)
                    if cache_file and len(labels) % 5000 < batch_size:
                        checkpoint_data = {
                            'rgb_data': np.concatenate(rgb_data, axis=0),
                            'brightness_data': np.concatenate(brightness_data, axis=0),
                            'labels': np.array(labels),
                            'processed': len(labels)
                        }
                        with open(f"{cache_file}.checkpoint", 'wb') as f:
                            pickle.dump(checkpoint_data, f)
                        print(f"   Checkpoint saved at sample {len(labels)}")

                except Exception as e:
                    print(f"⚠️ Error processing batch {batch_start}-{batch_end}: {e}")
                    continue

    except KeyboardInterrupt:
        print(f"\n⚠️ Processing interrupted at sample {len(labels)}")
        if len(labels) > 0:
            print("💾 Saving partial results...")
            # Concatenate what we have so far
            rgb_data = np.concatenate(rgb_data, axis=0)
            brightness_data = np.concatenate(brightness_data, axis=0)
            labels = np.array(labels)
            
            if cache_file:
                partial_data = {
                    'rgb_data': rgb_data,
                    'brightness_data': brightness_data,
                    'labels': labels,
                    'processed': len(labels)
                }
                with open(f"{cache_file}.partial", 'wb') as f:
                    pickle.dump(partial_data, f)
                print(f"   Partial results saved to {cache_file}.partial")
            
            return rgb_data, brightness_data, labels
        else:
            raise

    # Concatenate all batches
    if len(rgb_data) == 0:
        raise ValueError("No data was processed successfully")
        
    rgb_data = np.concatenate(rgb_data, axis=0)
    brightness_data = np.concatenate(brightness_data, axis=0)
    labels = np.array(labels)

    # Save to cache if specified
    if cache_file:
        final_data = {
            'rgb_data': rgb_data,
            'brightness_data': brightness_data,
            'labels': labels
        }
        with open(cache_file, 'wb') as f:
            pickle.dump(final_data, f)
        print(f"💾 Results cached to {cache_file}")

    print(f"✅ Conversion complete!")
    print(f"   RGB data shape: {rgb_data.shape}")
    print(f"   Brightness data shape: {brightness_data.shape}")
    print(f"   Labels shape: {labels.shape}")

    return rgb_data, brightness_data, labels

# Process training data
print("\n🚀 Processing training data...")
train_rgb, train_brightness, train_labels = convert_dataset_to_multi_stream_batch(
    train_dataset, 
    max_samples=5000,  # Reduce for faster demo
    cache_file="train_processed.pkl",
    batch_size=256  # Process 256 images at once
)

# Process test data
print("\n🧪 Processing test data...")
test_rgb, test_brightness, test_labels = convert_dataset_to_multi_stream_batch(
    test_dataset, 
    max_samples=1000,  # Reduce for faster demo
    cache_file="test_processed.pkl",
    batch_size=256  # Process 256 images at once
)

print(f"\n📊 Final Dataset Shapes:")
print(f"   Training RGB: {train_rgb.shape}")
print(f"   Training Brightness: {train_brightness.shape}")
print(f"   Training Labels: {train_labels.shape}")
print(f"   Test RGB: {test_rgb.shape}")
print(f"   Test Brightness: {test_brightness.shape}")
print(f"   Test Labels: {test_labels.shape}")

print("\n💡 Note: Processed data is cached. To reprocess, set force_reprocess=True")
print("🚀 Batch processing complete - much faster than image-by-image processing!")

# Final compatibility check (simplified to avoid NumPy 2.x issues)
print("\n🧪 Testing NumPy-PyTorch compatibility...")
try:
    # Simple test that should work with NumPy 2.x
    # Use explicit imports to avoid scoping issues
    import numpy
    import torch as pytorch
    
    test_data = [1, 2, 3]
    test_array = numpy.array(test_data, dtype=numpy.float32)
    test_tensor = pytorch.tensor(test_data, dtype=pytorch.float32)
    
    # Test basic operations
    array_sum = test_array.sum()
    tensor_sum = test_tensor.sum()
    
    # Test tensor conversion (this often fails with NumPy 2.x issues)
    converted_tensor = pytorch.from_numpy(test_array.copy())
    
    print("✅ NumPy-PyTorch integration working!")
    print("✅ Ready to proceed with CIFAR-100 loading!")
    
    # Note about NumPy 2.x warnings
    if numpy.__version__.startswith('2.'):
        print("ℹ️ Note: You may see NumPy 2.x compatibility warnings, but they don't affect functionality.")
        
except Exception as e:
    print(f"⚠️ NumPy-PyTorch compatibility issue: {e}")
    print("💡 This is likely a NumPy 2.x scoping issue, but shouldn't affect CIFAR-100 loading")
    print("✅ You can still proceed to Step 4 - the data loading will work fine!")

🔄 Starting RGB to RGBL data processing...
✅ RGBtoRGBL transform imported successfully

🚀 Processing training data...


NameError: name 'train_dataset' is not defined

In [ ]:
# Verify CIFAR-100 data structure
print("🔍 Verifying CIFAR-100 data structure...")

# Check a few samples
sample = train_dataset[0]
image, target = sample
print(f"✅ Sample structure: (image_tensor, integer_label)")
print(f"   Image shape: {image.shape}")
print(f"   Label: {target} ({cifar100_fine_labels[target]})")
print(f"   Label type: {type(target)}")

print(f"\n📊 Dataset summary:")
print(f"   Training samples: {len(train_dataset)}")
print(f"   Test samples: {len(test_dataset)}")
print(f"   Classes: 100 (fine labels 0-99)")
print("✅ Ready for multi-stream processing!")

## 6. Visualize Sample Images: RGB and Brightness Side by Side

Display sample images showing the original RGB and extracted brightness channels to understand the multi-stream transformation.

In [ ]:
def visualize_rgb_brightness_samples(rgb_data, brightness_data, labels, num_samples=5):
    """
    Visualize RGB and brightness images side by side.

    Args:
        rgb_data: RGB image data [N, 3, H, W]
        brightness_data: Brightness image data [N, 1, H, W]
        labels: Image labels
        num_samples: Number of samples to visualize
    """
    fig, axes = plt.subplots(num_samples, 2, figsize=(8, 2.5 * num_samples))
    fig.suptitle('RGB vs Brightness Channel Comparison', fontsize=16, fontweight='bold')

    for i in range(num_samples):
        # Get RGB image (convert from CHW to HWC for matplotlib)
        rgb_img = np.transpose(rgb_data[i], (1, 2, 0))

        # Get brightness image (squeeze channel dimension)
        brightness_img = brightness_data[i, 0]  # Remove channel dimension

        # Get class name
        class_name = cifar100_fine_labels[labels[i]]

        # Plot RGB image
        axes[i, 0].imshow(rgb_img)
        axes[i, 0].set_title(f'RGB - {class_name}', fontweight='bold')
        axes[i, 0].axis('off')

        # Plot brightness image
        axes[i, 1].imshow(brightness_img, cmap='gray')
        axes[i, 1].set_title(f'Brightness - {class_name}', fontweight='bold')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Visualize sample images
print("🖼️ Sample RGB vs Brightness Images:")
visualize_rgb_brightness_samples(train_rgb, train_brightness, train_labels, num_samples=5)

# Show data statistics
def show_data_statistics(rgb_data, brightness_data, labels):
    """Show basic statistics about the data."""
    print(f"\n📊 Data Statistics:")
    print(f"   RGB data range: [{rgb_data.min():.3f}, {rgb_data.max():.3f}]")
    print(f"   Brightness data range: [{brightness_data.min():.3f}, {brightness_data.max():.3f}]")
    print(f"   Number of unique classes: {len(np.unique(labels))}")

    # Class distribution
    unique_labels, counts = np.unique(labels, return_counts=True)
    print(f"   Samples per class: {counts.min()} - {counts.max()}")
    print(f"   Average samples per class: {counts.mean():.1f}")

show_data_statistics(train_rgb, train_brightness, train_labels)

## 7. Additional Data Visualizations

Explore the data with helpful visualizations including class distribution and pixel intensity analysis.

In [ ]:
# Class distribution visualization
def plot_class_distribution(labels, title="Class Distribution"):
    """Plot the distribution of classes in the dataset."""
    plt.figure(figsize=(12, 6))
    unique_labels, counts = np.unique(labels, return_counts=True)

    plt.bar(unique_labels, counts, alpha=0.7, color='skyblue', edgecolor='navy')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Class ID')
    plt.ylabel('Number of Samples')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Pixel intensity histograms
def plot_intensity_histograms(rgb_data, brightness_data):
    """Plot histograms of pixel intensities for RGB and brightness channels."""
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('Pixel Intensity Distributions', fontsize=16, fontweight='bold')

    # RGB histograms
    colors = ['red', 'green', 'blue']
    for i, color in enumerate(colors):
        axes[0, 0].hist(rgb_data[:, i].flatten(), bins=50, alpha=0.6,
                       color=color, label=f'{color.upper()} channel')
    axes[0, 0].set_title('RGB Channel Intensities')
    axes[0, 0].set_xlabel('Pixel Value')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Brightness histogram
    axes[0, 1].hist(brightness_data.flatten(), bins=50, alpha=0.7,
                   color='gray', edgecolor='black')
    axes[0, 1].set_title('Brightness Channel Intensities')
    axes[0, 1].set_xlabel('Pixel Value')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].grid(True, alpha=0.3)

    # Mean pixel values per channel
    rgb_means = np.mean(rgb_data, axis=(0, 2, 3))
    brightness_mean = np.mean(brightness_data)

    channel_names = ['Red', 'Green', 'Blue', 'Brightness']
    channel_means = [rgb_means[0], rgb_means[1], rgb_means[2], brightness_mean]

    axes[1, 0].bar(channel_names, channel_means,
                  color=['red', 'green', 'blue', 'gray'], alpha=0.7)
    axes[1, 0].set_title('Mean Pixel Values by Channel')
    axes[1, 0].set_ylabel('Mean Pixel Value')
    axes[1, 0].grid(True, alpha=0.3)

    # Sample grid
    axes[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Sample grid of images
def plot_sample_grid(rgb_data, labels, grid_size=(4, 8)):
    """Plot a grid of sample images."""
    fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=(16, 8))
    fig.suptitle('Sample Images from CIFAR-100 Dataset', fontsize=16, fontweight='bold')

    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            idx = i * grid_size[1] + j
            if idx < len(rgb_data):
                img = np.transpose(rgb_data[idx], (1, 2, 0))
                class_name = cifar100_fine_labels[labels[idx]]

                axes[i, j].imshow(img)
                axes[i, j].set_title(class_name, fontsize=8)
                axes[i, j].axis('off')

    plt.tight_layout()
    plt.show()

# Generate visualizations
print("📊 Generating additional visualizations...")

# Class distribution
plot_class_distribution(train_labels, "Training Set Class Distribution")

# Intensity histograms
plot_intensity_histograms(train_rgb[:1000], train_brightness[:1000])  # Sample for speed

# Sample grid
plot_sample_grid(train_rgb, train_labels)

## 8. Create Multi-Stream Neural Network Models

Create and configure multi-stream models using factory functions and the built-in compile API for clean, maintainable model setup.

**Key Features:**
- **Updated Factory Functions**: Support for different input sizes (`color_input_size`, `brightness_input_size`)
- **Built-in `.compile()` Method**: Keras-like model configuration with optimizer, loss, and metrics
- **Automatic Parameter Counting**: Easy model comparison and analysis
- **Device-Aware Initialization**: Automatic GPU detection and optimization
- **Forward Pass Testing**: Proper API usage validation with both research and classification modes

**Available Factory Functions:**
- **Dense Models**: `base_multi_channel_small`, `base_multi_channel_medium`, `base_multi_channel_large`
  - Now support: `color_input_size=3072, brightness_input_size=1024` for CIFAR-100
  - Backward compatible: `input_size=N` for same-size streams
- **CNN Models**: `multi_channel_resnet18`, `multi_channel_resnet34`, `multi_channel_resnet50`
  - Support different channel counts: `color_input_channels=3, brightness_input_channels=1`

**API Usage Examples:**
```python
# Dense model with different input sizes
model = base_multi_channel_medium(
    color_input_size=3072,      # RGB: 3*32*32
    brightness_input_size=1024, # Brightness: 1*32*32  
    num_classes=100
)

# CNN model with different channel counts
model = multi_channel_resnet18(
    color_input_channels=3,     # RGB channels
    brightness_input_channels=1, # Brightness channels
    num_classes=100
)

# Compile and use
model.compile(optimizer='adam', learning_rate=0.001)
model.fit(rgb_data, brightness_data, labels)
```

In [ ]:
# Model Configuration and Creation using Factory Functions
print("🏭 Creating Multi-Stream Neural Network Models using Factory Functions...")

# Check GPU availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {device}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Model configuration based on CIFAR-100 data
print(f"\n📊 Model Configuration:")
num_classes = 100  # CIFAR-100 has 100 classes
input_channels_rgb = 3  # RGB channels
input_channels_brightness = 1  # Single brightness channel
image_size = 32  # CIFAR-100 image size

# Calculate input sizes for dense model (flattened features)
rgb_input_size = input_channels_rgb * image_size * image_size  # 3 * 32 * 32 = 3072
brightness_input_size = input_channels_brightness * image_size * image_size  # 1 * 32 * 32 = 1024

print(f"   Number of classes: {num_classes}")
print(f"   RGB channels: {input_channels_rgb} (input size: {rgb_input_size})")
print(f"   Brightness channels: {input_channels_brightness} (input size: {brightness_input_size})")
print(f"   Image size: {image_size}x{image_size}")

# Import model classes and factory functions
try:
    from src.models.basic_multi_channel.base_multi_channel_network import (
        BaseMultiChannelNetwork, 
        base_multi_channel_small, 
        base_multi_channel_medium, 
        base_multi_channel_large
    )
    from src.models.basic_multi_channel.multi_channel_resnet_network import (
        MultiChannelResNetNetwork,
        multi_channel_resnet18,
        multi_channel_resnet34,
        multi_channel_resnet50
    )
    print("✅ Model classes and factory functions imported successfully")
except ImportError as e:
    print(f"❌ Failed to import model classes: {e}")
    print("💡 Make sure the project structure is correct and modules are available")
    raise

# Create Dense Network using factory function
print("\n🔬 Creating Dense Network using factory function...")
try:
    # Use the factory function with separate input sizes for multi-stream
    dense_model = base_multi_channel_medium(
        color_input_size=rgb_input_size,
        brightness_input_size=brightness_input_size,
        num_classes=num_classes,
        use_shared_classifier=True,  # Use shared classifier for efficiency
        activation='relu',
        device='auto'
    )
    
    # Compile the model using the built-in API
    print("⚙️ Compiling Dense Network...")
    dense_model.compile(
        optimizer='adam',
        learning_rate=0.001,
        loss='cross_entropy',
        metrics=['accuracy']
    )
    
    # Count parameters
    dense_params = sum(p.numel() for p in dense_model.parameters())
    dense_trainable = sum(p.numel() for p in dense_model.parameters() if p.requires_grad)
    
    print(f"✅ Dense Network created and compiled using factory function")
    print(f"   Architecture: Medium (512->256->128)")
    print(f"   Total parameters: {dense_params:,}")
    print(f"   Trainable parameters: {dense_trainable:,}")
    print(f"   RGB input size: {rgb_input_size} (flattened)")
    print(f"   Brightness input size: {brightness_input_size} (flattened)")
    
except Exception as e:
    print(f"❌ Failed to create Dense Network: {e}")
    print(f"💡 Error details: {str(e)}")
    import traceback
    traceback.print_exc()
    dense_model = None

# Create CNN Network using factory function
print("\n🔬 Creating CNN Network using factory function...")
try:
    # Use the factory function for ResNet-18 style CNN
    cnn_model = multi_channel_resnet18(
        num_classes=num_classes,
        color_input_channels=input_channels_rgb,
        brightness_input_channels=input_channels_brightness,
        activation='relu',
        device='auto'
    )
    
    # Compile the model using the built-in API
    print("⚙️ Compiling CNN Network...")
    cnn_model.compile(
        optimizer='adam',
        learning_rate=0.001,
        loss='cross_entropy',
        metrics=['accuracy']
    )
    
    # Count parameters
    cnn_params = sum(p.numel() for p in cnn_model.parameters())
    cnn_trainable = sum(p.numel() for p in cnn_model.parameters() if p.requires_grad)
    
    print(f"✅ CNN Network created and compiled using factory function")
    print(f"   Architecture: ResNet-18 style (2,2,2,2 blocks)")
    print(f"   Total parameters: {cnn_params:,}")
    print(f"   Trainable parameters: {cnn_trainable:,}")
    print(f"   Input shape: RGB {(input_channels_rgb, image_size, image_size)}, Brightness {(input_channels_brightness, image_size, image_size)}")
    
except Exception as e:
    print(f"❌ Failed to create CNN Network: {e}")
    print(f"💡 Error details: {str(e)}")
    cnn_model = None

# Model comparison
if dense_model is not None and cnn_model is not None:
    print(f"\n📈 Model Comparison:")
    print(f"   Dense Model: {dense_params:,} parameters")
    print(f"   CNN Model: {cnn_params:,} parameters")
    print(f"   CNN is {cnn_params/dense_params:.1f}x larger than Dense")
elif dense_model is not None:
    print(f"\n📈 Available Models:")
    print(f"   Dense Model: {dense_params:,} parameters")
elif cnn_model is not None:
    print(f"\n📈 Available Models:")
    print(f"   CNN Model: {cnn_params:,} parameters")

# Test model forward pass with sample data
print("\n🧪 Testing model forward pass with proper APIs...")

try:
    # Create sample batch data
    batch_size = 4
    sample_rgb = torch.randn(batch_size, input_channels_rgb, image_size, image_size).to(device)
    sample_brightness = torch.randn(batch_size, input_channels_brightness, image_size, image_size).to(device)
    
    print(f"   Sample RGB shape: {sample_rgb.shape}")
    print(f"   Sample brightness shape: {sample_brightness.shape}")
    
    # Test Dense Model
    if dense_model is not None:
        # Flatten inputs for dense model with correct sizes
        rgb_flat = sample_rgb.view(batch_size, rgb_input_size)  # Should be (4, 3072)
        brightness_flat = sample_brightness.view(batch_size, brightness_input_size)  # Should be (4, 1024)
        
        print(f"   Dense RGB flat shape: {rgb_flat.shape}")
        print(f"   Dense brightness flat shape: {brightness_flat.shape}")
        
        with torch.no_grad():
            # Test standard classification API
            dense_output = dense_model(rgb_flat, brightness_flat)
            print(f"✅ Dense model (forward_combined) output: {dense_output.shape}")
            
            # Test research API - check model's fusion type first
            if dense_model.use_shared_classifier:
                # Shared classifier returns single output from forward()
                dense_forward_output = dense_model.forward(rgb_flat, brightness_flat)
                print(f"✅ Dense model (forward, shared) output: {dense_forward_output.shape}")
            else:
                # Separate classifiers return tuple from forward()
                color_logits, brightness_logits = dense_model.forward(rgb_flat, brightness_flat)
                print(f"✅ Dense model (forward, separate) outputs: {color_logits.shape}, {brightness_logits.shape}")
    
    # Test CNN Model
    if cnn_model is not None:
        with torch.no_grad():
            # Test standard classification API
            cnn_output = cnn_model(sample_rgb, sample_brightness)
            print(f"✅ CNN model (forward_combined) output: {cnn_output.shape}")
            
            # Test research API  
            color_logits, brightness_logits = cnn_model.forward(sample_rgb, sample_brightness)
            print(f"✅ CNN model (forward) outputs: {color_logits.shape}, {brightness_logits.shape}")
    
    print("✅ All model tests passed!")
    
except Exception as e:
    print(f"❌ Model forward pass test failed: {e}")
    import traceback
    traceback.print_exc()

# Store available models for training
available_models = {}
if dense_model is not None:
    available_models['Dense Network'] = dense_model
if cnn_model is not None:
    available_models['CNN Network'] = cnn_model

if available_models:
    print(f"\n🎯 {len(available_models)} model(s) ready for training:")
    for model_name in available_models.keys():
        print(f"   ✅ {model_name}")
else:
    print("\n❌ No models available for training!")
    print("💡 Check the error messages above and fix the model creation issues")

print("\n🎯 Model creation complete! Models are compiled and ready for training.")

🏭 Creating Multi-Stream Neural Network Models using Factory Functions...


NameError: name 'torch' is not defined

## 9. Prepare Data for Training

Convert processed data to PyTorch tensors and create data loaders for efficient training.

In [ ]:
# Data Preparation for Training
print("📦 Preparing data for training...")

# Check if we have processed data
if 'train_rgb' not in locals() or 'train_brightness' not in locals():
    print("❌ No processed training data found!")
    print("💡 Please run the data processing cells first (Step 5)")
    raise ValueError("Training data not available")

print(f"✅ Found processed data:")
print(f"   Training RGB: {train_rgb.shape}")
print(f"   Training Brightness: {train_brightness.shape}")
print(f"   Training Labels: {train_labels.shape}")
print(f"   Test RGB: {test_rgb.shape}")
print(f"   Test Brightness: {test_brightness.shape}")
print(f"   Test Labels: {test_labels.shape}")

# Convert numpy arrays to PyTorch tensors
print("\n🔄 Converting to PyTorch tensors...")

# Training data
train_rgb_tensor = torch.FloatTensor(train_rgb)
train_brightness_tensor = torch.FloatTensor(train_brightness)
train_labels_tensor = torch.LongTensor(train_labels)

# Test data
test_rgb_tensor = torch.FloatTensor(test_rgb)
test_brightness_tensor = torch.FloatTensor(test_brightness)
test_labels_tensor = torch.LongTensor(test_labels)

print(f"✅ Tensors created:")
print(f"   Training RGB tensor: {train_rgb_tensor.shape}, dtype: {train_rgb_tensor.dtype}")
print(f"   Training brightness tensor: {train_brightness_tensor.shape}, dtype: {train_brightness_tensor.dtype}")
print(f"   Training labels tensor: {train_labels_tensor.shape}, dtype: {train_labels_tensor.dtype}")

# Normalize data to [0, 1] range if needed
if train_rgb_tensor.max() > 1.0:
    print("\n📊 Normalizing data to [0, 1] range...")
    train_rgb_tensor = train_rgb_tensor / 255.0
    train_brightness_tensor = train_brightness_tensor / 255.0
    test_rgb_tensor = test_rgb_tensor / 255.0
    test_brightness_tensor = test_brightness_tensor / 255.0
    print(f"✅ Data normalized: RGB range [{train_rgb_tensor.min():.3f}, {train_rgb_tensor.max():.3f}]")

# Create datasets
print("\n🗂️ Creating PyTorch datasets...")

class MultiStreamDataset(torch.utils.data.Dataset):
    """Custom dataset for multi-stream data (RGB + Brightness)"""
    
    def __init__(self, rgb_data, brightness_data, labels):
        self.rgb_data = rgb_data
        self.brightness_data = brightness_data
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'rgb': self.rgb_data[idx],
            'brightness': self.brightness_data[idx],
            'label': self.labels[idx]
        }

# Create dataset instances
train_dataset_multi = MultiStreamDataset(train_rgb_tensor, train_brightness_tensor, train_labels_tensor)
test_dataset_multi = MultiStreamDataset(test_rgb_tensor, test_brightness_tensor, test_labels_tensor)

print(f"✅ Datasets created:")
print(f"   Training dataset: {len(train_dataset_multi)} samples")
print(f"   Test dataset: {len(test_dataset_multi)} samples")

# Create data loaders
print("\n🚀 Creating data loaders...")

batch_size = 32  # Adjust based on GPU memory
num_workers = 2  # Adjust based on system

train_loader = torch.utils.data.DataLoader(
    train_dataset_multi,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=torch.cuda.is_available()
)

test_loader = torch.utils.data.DataLoader(
    test_dataset_multi,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=torch.cuda.is_available()
)

print(f"✅ Data loaders created:")
print(f"   Training batches: {len(train_loader)}")
print(f"   Test batches: {len(test_loader)}")
print(f"   Batch size: {batch_size}")

# Test data loader
print("\n🧪 Testing data loader...")
try:
    sample_batch = next(iter(train_loader))
    print(f"✅ Sample batch loaded:")
    print(f"   RGB batch shape: {sample_batch['rgb'].shape}")
    print(f"   Brightness batch shape: {sample_batch['brightness'].shape}")
    print(f"   Labels batch shape: {sample_batch['label'].shape}")
    print(f"   Labels range: {sample_batch['label'].min().item()} - {sample_batch['label'].max().item()}")
except Exception as e:
    print(f"❌ Data loader test failed: {e}")

print("\n📊 Data statistics:")
print(f"   Classes in training set: {len(torch.unique(train_labels_tensor))}")
print(f"   Classes in test set: {len(torch.unique(test_labels_tensor))}")
print(f"   RGB data range: [{train_rgb_tensor.min():.3f}, {train_rgb_tensor.max():.3f}]")
print(f"   Brightness data range: [{train_brightness_tensor.min():.3f}, {train_brightness_tensor.max():.3f}]")

print("\n✅ Data preparation complete! Ready for training.")

## 10. Train Multi-Stream Models

Train both the Dense and CNN models on the CIFAR-100 multi-stream data using the models' built-in `.fit()` API.

**Key Features:**
- Uses the models' built-in Keras-like `.fit()` method for clean, maintainable training
- Automatic optimization: batch size, workers, mixed precision based on device
- Built-in progress tracking and validation
- Proper input shape handling for Dense vs CNN models
- Consistent API across all model types

**API Usage:**
- `model.fit()` - Keras-like training API with automatic optimizations
- `model()` - Primary method for training, inference, and evaluation
- `model.forward()` - Research output (tuple of individual stream logits)

In [ ]:
# Training Configuration and Implementation
print("🚀 Setting up training configuration...")

# Training hyperparameters
num_epochs = 10  # Reduce for demo, increase for full training
learning_rate = 0.001
weight_decay = 1e-4

print(f"✅ Training Configuration:")
print(f"   Epochs: {num_epochs}")
print(f"   Learning rate: {learning_rate}")
print(f"   Weight decay: {weight_decay}")
print(f"   Device: {device}")

# Prepare data for model's .fit() method
# The models expect numpy arrays, so convert tensors back to numpy
train_rgb_np = train_rgb_tensor.cpu().numpy()
train_brightness_np = train_brightness_tensor.cpu().numpy()
train_labels_np = train_labels_tensor.cpu().numpy()

test_rgb_np = test_rgb_tensor.cpu().numpy()
test_brightness_np = test_brightness_tensor.cpu().numpy()
test_labels_np = test_labels_tensor.cpu().numpy()

print(f"\n📊 Data ready for training:")
print(f"   Training samples: {len(train_rgb_np)}")
print(f"   Test samples: {len(test_rgb_np)}")
print(f"   RGB input shape: {train_rgb_np.shape}")
print(f"   Brightness input shape: {train_brightness_np.shape}")

# Check if models are available
models_to_train = []

if 'available_models' in locals() and available_models:
    models_to_train = list(available_models.items())

if not models_to_train:
    print("❌ No models available for training!")
    print("💡 Please run the model creation cells first (Step 8)")
else:
    print(f"\n✅ Found {len(models_to_train)} models to train:")
    for name, _ in models_to_train:
        print(f"   - {name}")

print("\n🎯 Ready to start training using model's built-in .fit() API!")

In [ ]:
# Execute Training for All Models Using Built-in API
print("🚀 Starting model training using the models' built-in .fit() API...")

import time

# Store results for comparison
training_results = {}

# Train each model using their built-in .fit() method
for model_name, model in models_to_train:
    print(f"\n{'='*60}")
    print(f"🏋️ Training {model_name}")
    print(f"{'='*60}")
    
    try:
        start_time = time.time()
        
        # Prepare input data based on model type
        if 'Dense' in model_name:
            # Dense models expect flattened input
            rgb_input = train_rgb_np.reshape(train_rgb_np.shape[0], -1)
            brightness_input = train_brightness_np.reshape(train_brightness_np.shape[0], -1)
            val_rgb_input = test_rgb_np.reshape(test_rgb_np.shape[0], -1)
            val_brightness_input = test_brightness_np.reshape(test_brightness_np.shape[0], -1)
        else:
            # CNN models expect image-like input
            rgb_input = train_rgb_np
            brightness_input = train_brightness_np
            val_rgb_input = test_rgb_np
            val_brightness_input = test_brightness_np
        
        print(f"📊 Input shapes for {model_name}:")
        print(f"   RGB: {rgb_input.shape}")
        print(f"   Brightness: {brightness_input.shape}")
        
        # Train using the model's built-in .fit() method
        print(f"\n🔥 Training {model_name} using .fit() API...")
        model.fit(
            train_color_data=rgb_input,
            train_brightness_data=brightness_input,
            train_labels=train_labels_np,
            val_color_data=val_rgb_input,
            val_brightness_data=val_brightness_input,
            val_labels=test_labels_np,
            epochs=num_epochs,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            verbose=1  # Show progress bars
        )
        
        training_time = time.time() - start_time
        
        # Evaluate final accuracy using the model's built-in evaluation
        print(f"\n📈 Evaluating {model_name}...")
        model.eval()
        
        # Get predictions on test set
        with torch.no_grad():
            if 'Dense' in model_name:
                test_outputs = model(
                    torch.tensor(val_rgb_input, dtype=torch.float32).to(device),
                    torch.tensor(val_brightness_input, dtype=torch.float32).to(device)
                )
            else:
                test_outputs = model(
                    torch.tensor(test_rgb_np, dtype=torch.float32).to(device),
                    torch.tensor(test_brightness_np, dtype=torch.float32).to(device)
                )
            
            _, predicted = torch.max(test_outputs, 1)
            test_labels_tensor_device = torch.tensor(test_labels_np, dtype=torch.long).to(device)
            final_test_acc = (predicted == test_labels_tensor_device).float().mean().item() * 100
        
        # Store results
        training_results[model_name] = {
            'model': model,
            'final_test_acc': final_test_acc,
            'training_time': training_time,
        }
        
        print(f"✅ {model_name} training complete!")
        print(f"   Final test accuracy: {final_test_acc:.2f}%")
        print(f"   Training time: {training_time:.1f}s ({training_time/60:.1f} min)")
        
    except Exception as e:
        print(f"❌ Training failed for {model_name}: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*60}")
print("🎉 All Training Complete!")
print(f"{'='*60}")

# Display final results
if training_results:
    print("\n📊 Final Results Summary:")
    print("-" * 50)
    
    for model_name, result in training_results.items():
        print(f"{model_name}:")
        print(f"  Final Test Accuracy: {result['final_test_acc']:.2f}%")
        print(f"  Training Time: {result['training_time']:.1f}s ({result['training_time']/60:.1f} min)")
        print()
    
    # Find best model
    best_model_name = max(training_results.keys(), key=lambda k: training_results[k]['final_test_acc'])
    best_acc = training_results[best_model_name]['final_test_acc']
    
    print(f"🏆 Best Model: {best_model_name} ({best_acc:.2f}% accuracy)")
    
else:
    print("❌ No models were successfully trained!")

print("\n✅ Training phase complete using built-in model API!")

## 11. Training Results Visualization

Visualize the training progress and compare model performances.

In [ ]:
# Visualize Training Results
print("📊 Visualizing training results...")

def plot_model_comparison(training_results):
    """Create comparison charts for final model performance."""
    if not training_results:
        print("❌ No training results to compare!")
        return
    
    model_names = list(training_results.keys())
    test_accuracies = [result['final_test_acc'] for result in training_results.values()]
    training_times = [result['training_time'] / 60 for result in training_results.values()]  # Convert to minutes
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')
    
    # Test Accuracy Comparison
    bars1 = ax1.bar(model_names, test_accuracies, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'][:len(model_names)])
    ax1.set_title('Final Test Accuracy', fontweight='bold')
    ax1.set_ylabel('Accuracy (%)')
    ax1.set_ylim(0, max(test_accuracies) * 1.1 if test_accuracies else 1)
    
    # Add value labels on bars
    for bar, acc in zip(bars1, test_accuracies):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + max(test_accuracies) * 0.01,
                f'{acc:.2f}%', ha='center', va='bottom', fontweight='bold')
    
    ax1.grid(True, alpha=0.3)
    
    # Training Time Comparison
    bars2 = ax2.bar(model_names, training_times, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'][:len(model_names)])
    ax2.set_title('Training Time', fontweight='bold')
    ax2.set_ylabel('Time (minutes)')
    
    # Add value labels on bars
    for bar, time_val in zip(bars2, training_times):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + max(training_times) * 0.01,
                f'{time_val:.1f}m', ha='center', va='bottom', fontweight='bold')
    
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_efficiency_analysis(training_results):
    """Create efficiency analysis chart."""
    if not training_results:
        print("❌ No training results to analyze!")
        return
    
    model_names = list(training_results.keys())
    test_accuracies = [result['final_test_acc'] for result in training_results.values()]
    training_times = [result['training_time'] / 60 for result in training_results.values()]  # Convert to minutes
    
    # Calculate efficiency scores (accuracy per minute)
    efficiency_scores = [acc / time if time > 0 else 0 for acc, time in zip(test_accuracies, training_times)]
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    fig.suptitle('Model Efficiency Analysis (Accuracy per Minute)', fontsize=16, fontweight='bold')
    
    bars = ax.bar(model_names, efficiency_scores, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'][:len(model_names)])
    ax.set_title('Efficiency Score (Accuracy % per Minute)', fontweight='bold')
    ax.set_ylabel('Efficiency Score')
    
    # Add value labels on bars
    for bar, score in zip(bars, efficiency_scores):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + max(efficiency_scores) * 0.01,
                f'{score:.2f}', ha='center', va='bottom', fontweight='bold')
    
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Generate visualizations if we have training results
if 'training_results' in locals() and training_results:
    print("📊 Generating model comparison charts...")
    plot_model_comparison(training_results)
    
    print("\n🎯 Generating efficiency analysis...")
    plot_efficiency_analysis(training_results)
    
    # Print detailed comparison
    print("\n📋 Detailed Model Comparison:")
    print("-" * 70)
    print(f"{'Model Name':<20} {'Test Acc (%)':<12} {'Time (min)':<12} {'Parameters':<15}")
    print("-" * 70)
    
    for model_name, result in training_results.items():
        model = result['model']
        total_params = sum(p.numel() for p in model.parameters())
        time_min = result['training_time'] / 60
        
        print(f"{model_name:<20} {result['final_test_acc']:<12.2f} {time_min:<12.1f} {total_params:<15,}")
    
    print("-" * 70)
    
    # Efficiency analysis
    print("\n🎯 Efficiency Analysis:")
    best_acc_model = max(training_results.keys(), key=lambda k: training_results[k]['final_test_acc'])
    fastest_model = min(training_results.keys(), key=lambda k: training_results[k]['training_time'])
    
    print(f"   🏆 Best Accuracy: {best_acc_model} ({training_results[best_acc_model]['final_test_acc']:.2f}%)")
    print(f"   ⚡ Fastest Training: {fastest_model} ({training_results[fastest_model]['training_time']/60:.1f} min)")
    
    # Calculate efficiency score (accuracy per minute)
    efficiency_scores = {}
    for model_name, result in training_results.items():
        efficiency = result['final_test_acc'] / (result['training_time'] / 60)
        efficiency_scores[model_name] = efficiency
    
    most_efficient = max(efficiency_scores.keys(), key=lambda k: efficiency_scores[k])
    print(f"   🎯 Most Efficient: {most_efficient} ({efficiency_scores[most_efficient]:.2f} acc%/min)")
    
else:
    print("❌ No training results available for visualization!")
    print("💡 Make sure to run the training cells first (Step 10)")

print("\n✅ Training results visualization complete!")

## 12. Model Evaluation and Analysis

Perform detailed evaluation including confusion matrix, per-class accuracy, and error analysis.

In [ ]:
# Comprehensive Model Evaluation
print("🔍 Performing comprehensive model evaluation...")

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns

def evaluate_model_detailed(model, model_name, test_loader, device, class_names):
    """
    Perform detailed evaluation of a trained model.
    
    Args:
        model: Trained model
        model_name: Name of the model for display
        test_loader: Test data loader
        device: Device to run evaluation on
        class_names: List of class names for CIFAR-100
    
    Returns:
        Dictionary with evaluation metrics and predictions
    """
    print(f"\n🔬 Evaluating {model_name}...")
    
    model.eval()
    all_predictions = []
    all_targets = []
    all_probabilities = []
    
    with torch.no_grad():
        test_pbar = tqdm(test_loader, desc=f"Evaluating {model_name}")
        
        for batch in test_pbar:
            rgb_data = batch['rgb'].to(device)
            brightness_data = batch['brightness'].to(device)
            targets = batch['label'].to(device)
            
            # Forward pass
            if 'Dense' in model_name:
                rgb_flat = rgb_data.view(rgb_data.size(0), -1)
                brightness_flat = brightness_data.view(brightness_data.size(0), -1)
                outputs = model(rgb_flat, brightness_flat)
            else:
                outputs = model(rgb_data, brightness_data)
            
            # Get predictions and probabilities
            probabilities = torch.softmax(outputs, dim=1)
            _, predictions = torch.max(outputs, 1)
            
            # Store results
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    # Convert to numpy arrays
    predictions = np.array(all_predictions)
    targets = np.array(all_targets)
    probabilities = np.array(all_probabilities)
    
    # Calculate metrics
    accuracy = accuracy_score(targets, predictions)
    precision = precision_score(targets, predictions, average='weighted', zero_division=0)
    recall = recall_score(targets, predictions, average='weighted', zero_division=0)
    f1 = f1_score(targets, predictions, average='weighted', zero_division=0)
    
    print(f"✅ {model_name} Evaluation Complete:")
    print(f"   Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall: {recall:.4f}")
    print(f"   F1 Score: {f1:.4f}")
    
    return {
        'model_name': model_name,
        'predictions': predictions,
        'targets': targets,
        'probabilities': probabilities,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

def plot_confusion_matrix(evaluation_result, class_names, figsize=(12, 10)):
    """Plot confusion matrix for model evaluation."""
    predictions = evaluation_result['predictions']
    targets = evaluation_result['targets']
    model_name = evaluation_result['model_name']
    
    # Calculate confusion matrix
    cm = confusion_matrix(targets, predictions)
    
    # Normalize confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Plot
    plt.figure(figsize=figsize)
    sns.heatmap(cm_normalized, annot=False, cmap='Blues', fmt='.2f',
                xticklabels=False, yticklabels=False)
    plt.title(f'Confusion Matrix - {model_name}\nAccuracy: {evaluation_result["accuracy"]:.3f}', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Predicted Class')
    plt.ylabel('True Class')
    plt.tight_layout()
    plt.show()

def analyze_per_class_performance(evaluation_result, class_names, top_k=10):
    """Analyze per-class performance and show best/worst performing classes."""
    predictions = evaluation_result['predictions']
    targets = evaluation_result['targets']
    model_name = evaluation_result['model_name']
    
    # Calculate per-class accuracy
    per_class_acc = []
    per_class_counts = []
    
    for class_id in range(len(class_names)):
        class_mask = targets == class_id
        if class_mask.sum() > 0:
            class_predictions = predictions[class_mask]
            class_targets = targets[class_mask]
            class_accuracy = (class_predictions == class_targets).mean()
            per_class_acc.append(class_accuracy)
            per_class_counts.append(class_mask.sum())
        else:
            per_class_acc.append(0.0)
            per_class_counts.append(0)
    
    per_class_acc = np.array(per_class_acc)
    per_class_counts = np.array(per_class_counts)
    
    # Sort by accuracy
    sorted_indices = np.argsort(per_class_acc)
    
    print(f"\n📊 Per-Class Performance Analysis - {model_name}")
    print("=" * 60)
    
    # Best performing classes
    print(f"\n🏆 Top {top_k} Best Performing Classes:")
    print("-" * 50)
    for i in range(-1, -top_k-1, -1):
        idx = sorted_indices[i]
        class_name = class_names[idx]
        accuracy = per_class_acc[idx] * 100
        count = per_class_counts[idx]
        print(f"{-i:2d}. {class_name:<20} {accuracy:6.2f}% ({count:3d} samples)")
    
    # Worst performing classes
    print(f"\n💥 Top {top_k} Worst Performing Classes:")
    print("-" * 50)
    for i in range(top_k):
        idx = sorted_indices[i]
        class_name = class_names[idx]
        accuracy = per_class_acc[idx] * 100
        count = per_class_counts[idx]
        print(f"{i+1:2d}. {class_name:<20} {accuracy:6.2f}% ({count:3d} samples)")
    
    # Overall statistics
    print(f"\n📈 Overall Statistics:")
    print(f"   Mean per-class accuracy: {per_class_acc.mean()*100:.2f}%")
    print(f"   Std per-class accuracy: {per_class_acc.std()*100:.2f}%")
    print(f"   Best class accuracy: {per_class_acc.max()*100:.2f}%")
    print(f"   Worst class accuracy: {per_class_acc.min()*100:.2f}%")
    
    return per_class_acc, per_class_counts

def plot_class_performance_distribution(evaluation_results, class_names):
    """Plot distribution of per-class accuracies for all models."""
    if not evaluation_results:
        print("❌ No evaluation results to plot!")
        return
    
    plt.figure(figsize=(14, 8))
    
    for i, (model_name, eval_result) in enumerate(evaluation_results.items()):
        per_class_acc, _ = analyze_per_class_performance(eval_result, class_names, top_k=5)
        
        # Plot histogram
        plt.subplot(2, len(evaluation_results), i + 1)
        plt.hist(per_class_acc * 100, bins=20, alpha=0.7, edgecolor='black')
        plt.title(f'{model_name}\nPer-Class Accuracy Distribution')
        plt.xlabel('Accuracy (%)')
        plt.ylabel('Number of Classes')
        plt.grid(True, alpha=0.3)
        
        # Plot box plot
        plt.subplot(2, len(evaluation_results), len(evaluation_results) + i + 1)
        plt.boxplot(per_class_acc * 100, vert=True)
        plt.title(f'{model_name}\nAccuracy Box Plot')
        plt.ylabel('Accuracy (%)')
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Perform evaluation if we have trained models
evaluation_results = {}

if 'training_results' in locals() and training_results:
    print("🔍 Starting comprehensive evaluation...")
    
    for model_name, training_result in training_results.items():
        model = training_result['model']
        
        try:
            eval_result = evaluate_model_detailed(
                model=model,
                model_name=model_name,
                test_loader=test_loader,
                device=device,
                class_names=cifar100_fine_labels
            )
            evaluation_results[model_name] = eval_result
            
            # Plot confusion matrix for each model
            print(f"\n📊 Generating confusion matrix for {model_name}...")
            plot_confusion_matrix(eval_result, cifar100_fine_labels, figsize=(10, 8))
            
        except Exception as e:
            print(f"❌ Evaluation failed for {model_name}: {e}")
            continue
    
    # Generate comparison plots
    if evaluation_results:
        print("\n📈 Generating per-class performance analysis...")
        plot_class_performance_distribution(evaluation_results, cifar100_fine_labels)
        
        # Compare models side by side
        print("\n🔄 Model Performance Comparison:")
        print("=" * 80)
        print(f"{'Model':<20} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
        print("=" * 80)
        
        for model_name, eval_result in evaluation_results.items():
            print(f"{model_name:<20} {eval_result['accuracy']:<10.4f} {eval_result['precision']:<10.4f} "
                  f"{eval_result['recall']:<10.4f} {eval_result['f1_score']:<10.4f}")
        
        print("=" * 80)
        
    else:
        print("❌ No models were successfully evaluated!")
        
else:
    print("❌ No trained models available for evaluation!")
    print("💡 Make sure to run the training cells first (Step 10)")

print("\n✅ Model evaluation complete!")

## 13. Model Saving and Inference Demo

Save trained models and demonstrate inference on new samples.

In [ ]:
# Model Saving and Inference Demo
print("💾 Setting up model saving and inference...")

import os
from pathlib import Path

def save_model(model, model_name, training_result, save_dir="models"):
    """
    Save a trained model with its metadata.
    
    Args:
        model: Trained PyTorch model
        model_name: Name of the model
        training_result: Training results dictionary
        save_dir: Directory to save models
    """
    # Create save directory
    save_path = Path(save_dir)
    save_path.mkdir(exist_ok=True)
    
    # Prepare model info
    model_info = {
        'model_name': model_name,
        'final_test_accuracy': training_result['final_test_acc'],
        'training_time': training_result['training_time'],
        'model_state_dict': model.state_dict(),
        'model_class': model.__class__.__name__,
        'num_parameters': sum(p.numel() for p in model.parameters()),
        'training_history': training_result['history']
    }
    
    # Save model
    model_file = save_path / f"{model_name.replace(' ', '_').lower()}_cifar100.pth"
    torch.save(model_info, model_file)
    
    print(f"✅ {model_name} saved to: {model_file}")
    return model_file

def load_model(model_file, model_class, device):
    """
    Load a saved model.
    
    Args:
        model_file: Path to saved model file
        model_class: Model class to instantiate
        device: Device to load model on
    
    Returns:
        Loaded model and metadata
    """
    checkpoint = torch.load(model_file, map_location=device)
    
    # Print model info
    print(f"📋 Model Info:")
    print(f"   Name: {checkpoint['model_name']}")
    print(f"   Class: {checkpoint['model_class']}")
    print(f"   Test Accuracy: {checkpoint['final_test_accuracy']:.2f}%")
    print(f"   Parameters: {checkpoint['num_parameters']:,}")
    print(f"   Training Time: {checkpoint['training_time']/60:.1f} minutes")
    
    return checkpoint

def demonstrate_inference(model, model_name, test_loader, device, class_names, num_samples=8):
    """
    Demonstrate model inference on random test samples.
    
    Args:
        model: Trained model
        model_name: Name of the model
        test_loader: Test data loader
        device: Device to run inference on
        class_names: List of class names
        num_samples: Number of samples to demonstrate
    """
    print(f"\n🎯 Demonstrating {model_name} inference...")
    
    model.eval()
    
    # Get a batch of test data
    test_batch = next(iter(test_loader))
    rgb_data = test_batch['rgb'][:num_samples].to(device)
    brightness_data = test_batch['brightness'][:num_samples].to(device)
    true_labels = test_batch['label'][:num_samples]
    
    # Make predictions
    with torch.no_grad():
        if 'Dense' in model_name:
            rgb_flat = rgb_data.view(rgb_data.size(0), -1)
            brightness_flat = brightness_data.view(brightness_data.size(0), -1)
            outputs = model(rgb_flat, brightness_flat)
        else:
            outputs = model(rgb_data, brightness_data)
        
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted_labels = torch.max(outputs, 1)
    
    # Visualize results
    fig, axes = plt.subplots(2, num_samples//2, figsize=(16, 8))
    fig.suptitle(f'{model_name} - Inference Demo', fontsize=16, fontweight='bold')
    
    axes = axes.flatten()
    
    for i in range(num_samples):
        # Get RGB image for display
        rgb_img = rgb_data[i].cpu().numpy().transpose(1, 2, 0)
        
        # Get predictions
        true_class = class_names[true_labels[i].item()]
        pred_class = class_names[predicted_labels[i].item()]
        confidence = probabilities[i][predicted_labels[i]].item() * 100
        
        # Determine color (green for correct, red for incorrect)
        color = 'green' if true_labels[i] == predicted_labels[i] else 'red'
        
        # Plot
        axes[i].imshow(rgb_img)
        axes[i].set_title(f'True: {true_class}\nPred: {pred_class}\nConf: {confidence:.1f}%', 
                         color=color, fontweight='bold', fontsize=10)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate accuracy for this batch
    batch_accuracy = (predicted_labels.cpu() == true_labels).float().mean().item() * 100
    print(f"   Batch accuracy: {batch_accuracy:.1f}%")
    
    return predicted_labels.cpu().numpy(), probabilities.cpu().numpy()

# Save all trained models
saved_models = {}

if 'training_results' in locals() and training_results:
    print("💾 Saving trained models...")
    
    for model_name, training_result in training_results.items():
        try:
            model_file = save_model(
                model=training_result['model'],
                model_name=model_name,
                training_result=training_result
            )
            saved_models[model_name] = model_file
        except Exception as e:
            print(f"❌ Failed to save {model_name}: {e}")
    
    print(f"\n✅ Saved {len(saved_models)} models to 'models/' directory")
    
    # Demonstrate inference for each model
    print("\n🎯 Running inference demonstrations...")
    
    for model_name, training_result in training_results.items():
        try:
            model = training_result['model']
            predictions, probabilities = demonstrate_inference(
                model=model,
                model_name=model_name,
                test_loader=test_loader,
                device=device,
                class_names=cifar100_fine_labels,
                num_samples=8
            )
        except Exception as e:
            print(f"❌ Inference demo failed for {model_name}: {e}")
            continue
    
else:
    print("❌ No trained models available for saving!")
    print("💡 Make sure to run the training cells first (Step 10)")

# Example of how to load a saved model (for future use)
print("\n📖 Example: Loading a saved model (for future use)")
print("```python")
print("# To load a model in the future:")
print("checkpoint = torch.load('models/dense_network_cifar100.pth')")
print("model = BaseMultiChannelNetwork(...)  # Initialize with same parameters")
print("model.load_state_dict(checkpoint['model_state_dict'])")
print("model.eval()")
print("```")

print("\n✅ Model saving and inference demo complete!")

## 14. Conclusion and Summary

Summary of results, key findings, and next steps for the multi-stream neural network project.

In [ ]:
# 🎉 Multi-Stream Neural Networks: Project Summary
print("📋 Generating project summary...")

def generate_project_summary():
    """Generate a comprehensive summary of the project results."""
    
    print("🎯 MULTI-STREAM NEURAL NETWORKS ON CIFAR-100")
    print("=" * 60)
    
    print("\n📊 PROJECT OVERVIEW:")
    print("   • Dataset: CIFAR-100 (100 classes, 32x32 images)")
    print("   • Architecture: Multi-stream (RGB + Brightness channels)")
    print("   • Models: Dense Network vs CNN (ResNet-style)")
    print("   • Training: Multi-channel data with batch processing")
    print("   • Evaluation: Comprehensive analysis with visualizations")
    
    if 'training_results' in locals() and training_results:
        print("\n🏆 TRAINING RESULTS:")
        print("-" * 40)
        
        best_model = None
        best_accuracy = 0
        
        for model_name, result in training_results.items():
            accuracy = result['final_test_acc']
            time_min = result['training_time'] / 60
            params = sum(p.numel() for p in result['model'].parameters())
            
            print(f"   {model_name}:")
            print(f"     • Test Accuracy: {accuracy:.2f}%")
            print(f"     • Training Time: {time_min:.1f} minutes")
            print(f"     • Parameters: {params:,}")
            print(f"     • Efficiency: {accuracy/time_min:.2f} acc%/min")
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model_name
            
            print()
        
        print(f"🏅 BEST MODEL: {best_model} ({best_accuracy:.2f}% accuracy)")
        
        # Architecture comparison
        if len(training_results) > 1:
            print("\n🔬 ARCHITECTURE ANALYSIS:")
            print("-" * 40)
            models = list(training_results.items())
            
            if len(models) == 2:
                model1_name, model1_result = models[0]
                model2_name, model2_result = models[1]
                
                acc_diff = abs(model1_result['final_test_acc'] - model2_result['final_test_acc'])
                time_diff = abs(model1_result['training_time'] - model2_result['training_time']) / 60
                
                print(f"   • Accuracy difference: {acc_diff:.2f}%")
                print(f"   • Training time difference: {time_diff:.1f} minutes")
                
                if 'Dense' in model1_name or 'Dense' in model2_name:
                    print("   • Dense vs CNN comparison completed")
                    if acc_diff < 2.0:
                        print("   • Both architectures show similar performance")
                    else:
                        winner = model1_name if model1_result['final_test_acc'] > model2_result['final_test_acc'] else model2_name
                        print(f"   • {winner} shows superior performance")
    
    else:
        print("\n⚠️ No training results available for summary")
    
    print("\n🔧 TECHNICAL ACHIEVEMENTS:")
    print("-" * 40)
    print("   ✅ Modular CIFAR-100 data loading and preprocessing")
    print("   ✅ RGB to RGBL transformation with batch processing")
    print("   ✅ Multi-stream neural network architectures")
    print("   ✅ Efficient training pipeline with GPU acceleration")
    print("   ✅ Comprehensive evaluation and visualization")
    print("   ✅ Model saving and inference demonstration")
    print("   ✅ Production-ready code structure")
    
    print("\n🚀 NEXT STEPS & IMPROVEMENTS:")
    print("-" * 40)
    print("   • Scale training to full CIFAR-100 dataset (50k training samples)")
    print("   • Implement advanced techniques:")
    print("     - Data augmentation (rotation, flip, crop)")
    print("     - Learning rate scheduling and early stopping")
    print("     - Model ensembling")
    print("     - Attention mechanisms")
    print("   • Experiment with different brightness extraction methods")
    print("   • Add more sophisticated CNN architectures (ResNet-50, EfficientNet)")
    print("   • Hyperparameter optimization (learning rate, batch size, etc.)")
    print("   • Transfer learning from pre-trained models")
    print("   • Multi-GPU training for faster convergence")
    
    print("\n💡 KEY INSIGHTS:")
    print("-" * 40)
    print("   • Multi-stream processing effectively utilizes RGB and brightness")
    print("   • Batch processing significantly improves data preprocessing speed")
    print("   • Both dense and CNN architectures show promise for multi-stream data")
    print("   • Modular design enables easy experimentation and extension")
    print("   • CIFAR-100's 100 classes provide good complexity for evaluation")
    
    print("\n📚 RESOURCES & DOCUMENTATION:")
    print("-" * 40)
    print("   • Code: src/ directory with modular components")
    print("   • Models: Saved in models/ directory")
    print("   • Tests: tests/ directory with comprehensive test suite")
    print("   • Documentation: README.md and inline documentation")
    print("   • Results: Cached processed data and training outputs")
    
    print("\n🎯 PROJECT STATUS: COMPLETE ✅")
    print("   Ready for production use and further research!")

# Run the summary
generate_project_summary()

print("\n" + "="*60)
print("🙏 THANK YOU FOR EXPLORING MULTI-STREAM NEURAL NETWORKS!")
print("="*60)
print("\n💬 Questions or improvements? Check the GitHub repository:")
print("   https://github.com/clingergab/Multi-Stream-Neural-Networks")
print("\n🚀 Happy experimenting with multi-stream architectures!")